In [27]:
## Définition des fonctons 

import os
import re
import time
import json


from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.firefox import options as firefox_options


def get_driver(headless=False):
    """Returns a selenium firefox webdriver
    Args:
        headless (bool, optional): Whether to hide the browser. Defaults to False.
    Returns:
        selenium.webdriver.Firefox: firefox webdriver
    """
    if headless:
        options = firefox_options.Options()
        options.add_argument("--headless")
        driver = webdriver.Firefox(options=options)
    else:
        driver = webdriver.Firefox()
    return driver

def get_page(count=10,headless=False):
  driver = get_driver(headless=headless)

  pages = []

  for page_number in range(1, count+1):
    #page_url = f"https://www.logic-immo.com/vente-immobilier/options/grouplocalities=1_0,21_0,12_0,8_0,84_1,11_0,13_0/groupprptypesids=1,2/page={page_number}"
    page_url = f"https://www.seloger.com/list.htm?projects=2%2C5&types=2%2C1&natures=1%2C2%2C4&places=%5B%7B%22divisions%22%3A%5B2238%5D%7D%5D&enterprise=0&qsVersion=1.0&LISTING-LISTpg={page_number}"
    #page_url = f"https://www.leboncoin.fr/_immobilier_/offres/p-{page_number}"
    driver.get(page_url)
    time.sleep(13)
    pages.append(driver.page_source.encode("utf-8"))
  return pages

def save_pages(pages):
  os.makedirs("data", exist_ok=True)
  for page_nb, page in enumerate(pages):
    with open(f"data/page={page_nb}.html", "wb") as f_out: 
      f_out.write(page)

def parse_pages():
  pages_paths = os.listdir("data")

  results = pd.DataFrame()
  results = pd.DataFrame()
  pages_paths = os.listdir("data")
  for pages_path in pages_paths:
      with open(os.path.join("data", pages_path), "rb") as f_in:
          page = f_in.read().decode("utf-8")
          results = results.append(parse_page(page))
  return results

def parse_page(page):

  soup = BeautifulSoup(page,"html.parser")
  result = pd.DataFrame()

  result["price (€)"] = [
     clean_price(tag) for tag in soup.find_all(attrs={"class": "announceDtlPrice"})
  ]

  result["type"] = [
      clean_type(tag) for tag in soup.find_all(attrs={"class": "announceDtlInfosPropertyType"})
  ]

  result["adresse"] = [
      clean_postal_code(tag) for tag in soup.find_all(attrs={"class": "announcePropertyLocation"})
  ]

  areas = soup.find_all(attrs={"class": "announceDtlInfosContent"})
  result["description"] = [tag.text.strip() for tag in areas]
     

  # areas = soup.find_all(attrs={"class": "announceDtlPrice"})
  # areas = soup.find_all(attrs={"class": "Price__PriceContainer-sc-1g9fitq-0 knHjrC"})
  # #areas = soup.find_all(attrs={"class": "sc-fHxwqH hAXnvi"})
  # result["price"] = [tag.text.strip() for tag in areas]
  return result

def clean_price(tag):
    text = tag.text.strip()
    price = int(text.replace("€", "").replace(" ", "").replace("\xa0", ""))
    return price

def clean_type(tag):
    text = tag.text.strip()
    return text.replace("Location ", "")


def clean_surface(tag):
    text = tag.text.strip()
    return int(text.replace("m²", ""))


def clean_rooms(tag):
    text = tag.text.strip()
    rooms = int(text.replace("p.", "").replace(" ", "")) 
    return rooms


def clean_postal_code(tag):
    text = tag.text.strip()
    match = re.match(".*\(([0-9]+)\).*", text)
    return match.groups()[0]


def main():
  #pages = get_page()
  #save_pages(pages)
  results = parse_pages()
  return results
main()
url = "https://www.logic-immo.com/vente-immobilier/options/grouplocalities=1_0,21_0,12_0,8_0,84_1,11_0,13_0/groupprptypesids=1,2/page=1"

C:\Users\Admin\AppData\Local\Temp\ipykernel_29608\2764442771.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(parse_page(page))
C:\Users\Admin\AppData\Local\Temp\ipykernel_29608\2764442771.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(parse_page(page))
C:\Users\Admin\AppData\Local\Temp\ipykernel_29608\2764442771.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(parse_page(page))
C:\Users\Admin\AppData\Local\Temp\ipykernel_29608\2764442771.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(parse_page(page))
C:\Users\Admin\A

In [28]:
#On récupére les données et affichage des données 
results = main()
print(results)

C:\Users\Admin\AppData\Local\Temp\ipykernel_29608\2764442771.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(parse_page(page))
C:\Users\Admin\AppData\Local\Temp\ipykernel_29608\2764442771.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(parse_page(page))
C:\Users\Admin\AppData\Local\Temp\ipykernel_29608\2764442771.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(parse_page(page))
C:\Users\Admin\AppData\Local\Temp\ipykernel_29608\2764442771.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(parse_page(page))
C:\Users\Admin\A

    price (€)                  type adresse  \
0    264600.0  Appartement à vendre   92140   
1    264775.0       Studio à vendre   85100   
2    265000.0  Appartement à vendre   06220   
3    265000.0  Appartement à vendre   83400   
4    269000.0  Appartement à vendre   06220   
..        ...                   ...     ...   
15  1100000.0  Appartement à vendre   75116   
16  1120000.0       Maison à vendre   44700   
17  1125000.0  Appartement à vendre   13100   
18  1130000.0       Maison à vendre   83440   
19  1130000.0  Appartement à vendre   13007   

                            description  
0    Appartement à vendre 63m² 3 pièces  
1          Studio à vendre 24m² 1 pièce  
2    Appartement à vendre 39m² 2 pièces  
3    Appartement à vendre 45m² 2 pièces  
4    Appartement à vendre 80m² 4 pièces  
..                                  ...  
15   Appartement à vendre 54m² 2 pièces  
16       Maison à vendre 248m² 7 pièces  
17   Appartement à vendre 97m² 4 pièces  
18       Maison

C:\Users\Admin\AppData\Local\Temp\ipykernel_29608\2764442771.py:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(parse_page(page))


In [29]:
#On affiche les données à séparer 
results['description']

0      Appartement à vendre 63m² 3 pièces
1            Studio à vendre 24m² 1 pièce
2      Appartement à vendre 39m² 2 pièces
3      Appartement à vendre 45m² 2 pièces
4      Appartement à vendre 80m² 4 pièces
                     ...                 
15     Appartement à vendre 54m² 2 pièces
16         Maison à vendre 248m² 7 pièces
17     Appartement à vendre 97m² 4 pièces
18         Maison à vendre 200m² 8 pièces
19    Appartement à vendre 124m² 3 pièces
Name: description, Length: 9920, dtype: object

In [31]:
def extract_info(description):
    # Initialiser les variables
    nb_pieces = ""
    nb_chambres = ""
    etage = ""
    ascenseur = ""
    parking = ""
    jardin = ""
    terrasse = ""
    surface = ""
    
    # Extraire les informations de la colonne "description" à l'aide des expressions régulières
    matches = re.findall(r'(\d+)\s*pièces|(\d+)\s*chambre*|(\d+)\s*m²|Étage\s*(\d+)/|(Ascenseur)|(Parking)|(Jardin)|(Terrasse)', description)
    # Affecter les valeurs extraites aux variables correspondantes
    for match in matches:
        if match[0]:
            nb_pieces = match[0]
        elif match[1]:
            nb_chambres = match[1]
        elif match[2]:
            surface = match[2]     
        elif match[3]:
            etage = match[3]
    
    for match in matches:
      if match[4]:
        ascenseur = "True"
      else: 
        ascenseur = "false"
    
    for match in matches:
      if match[5]:
        parking = "True"
      else: 
        parking = "false"
    for match in matches:
      if match[6]:
        jardin = "True"
      else: 
        jardin =  "false"
    for match in matches:
      if match[7]:
        terrasse = "True"
      else: 
        terrasse = "false"
    
    # Retourner les valeurs extraites sous forme d'un dictionnaire
    return {"nb_pieces": nb_pieces, "nb_chambres": nb_chambres, "surface" : surface , "etage": etage, "ascenseur": ascenseur, "parking": parking, "jardin": jardin, "terrasse": terrasse}

In [32]:
# Appliquer la fonction extract_info à la colonne "description" pour créer les nouvelles colonnes
results=pd.concat([results,results['description'].apply(lambda x: pd.Series(extract_info(x)))], axis=1)


# Supprimer la colonne "description" originale
results.drop("description", axis=1, inplace=True)

# On renomme la colonne "adresse" par "code_postal"
results = results.rename(columns={ 'adresse': 'code_postal' })

In [33]:
## Affichage du DataFrame aprés nettoyage de données 
results

,price (€),type,code_postal,nb_pieces,nb_chambres,surface,etage,ascenseur,parking,jardin,terrasse
0,264600.0,Appartement à vendre,92140,3,,63,,false,false,false,false
1,264775.0,Studio à vendre,85100,,,24,,false,false,false,false
2,265000.0,Appartement à vendre,06220,2,,39,,false,false,false,false
3,265000.0,Appartement à vendre,83400,2,,45,,false,false,false,false
4,269000.0,Appartement à vendre,06220,4,,80,,false,false,false,false
...,...,...,...,...,...,...,...,...,...,...,...
15,1100000.0,Appartement à vendre,75116,2,,54,,false,false,false,false
16,1120000.0,Maison à vendre,44700,7,,248,,false,false,false,false
17,1125000.0,Appartement à vendre,13100,4,,97,,false,false,false,false
18,1130000.0,Maison à vendre,83440,8,,200,,false,false,false,false


In [34]:
results.to_csv('data/houseScapeData.csv', index=False)

In [ ]:
## On voit qu'il y a différent type de "Maison"
print(results.type.value_counts())

Series([], Name: type, dtype: int64)


In [ ]:
## On regroupe tous les types de maison ensemble
dfb = results["type"]
type_de_bien = ["Maison 1 étage","Maison 2 étages","Maison 3 étages","Maison de ville 1 étage","Maison de ville","Maison de village","Maison en pierre 2 étages","Maison contemporaine"]
for x in type_de_bien:
    dfb = dfb.replace(x,"Maison")
dfb.value_counts()

Series([], Name: type, dtype: int64)

In [ ]:
## On regroupe tous les types de Duplex ensemble
type_de_bien = ["Duplex meublé"]
for x in type_de_bien:
    dfb = dfb.replace(x,"Duplex")
dfb.value_counts()

Series([], Name: type, dtype: int64)

In [ ]:
## On regroupe tous les types d'appart ensemble
type_de_bien = ["Appartement neuf"]
for x in type_de_bien:
    dfb = dfb.replace(x,"Appartement")
dfb.value_counts()

Series([], Name: type, dtype: int64)

In [ ]:
## Modification du type de "surface" afin de pouvoir exploiter la données
m2 = results["surface"].astype(str).astype(int)
## Ajout d'une colonne du prix au m2 arrondie au dixiéme
df_clean = results.assign(price_m2 = round(results["price (€)"]/m2,2))
## Mise à jour des valeurs "type"
df_clean["type"] = dfb

KeyError: 'surface'

In [ ]:
# Cette méthode crée la sortie d’une statistique complète de l’ensemble de données numériques.
#print(results.describe())
df_clean.describe()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Information sur le dataframe
results.info 

# Affichage des premieres donnees 

nb_annonce = results[['price (€)','code_postal']].groupby('code_postal').count().sort_values(by='price (€)', ascending=False)
nb_annonce.reset_index(0, inplace=True)
nb_annonce.rename(columns={'price (€)':'Nb_annonces'}, inplace=True)
nb_annonce.head()

In [ ]:
plt.figure(figsize=(50,25))
sns.barplot(x=nb_annonce['code_postal'], y=nb_annonce['Nb_annonces'], palette="summer")
plt.xlabel('\nCode Postal', fontsize=50, color='#c0392b')
plt.ylabel("Nombre d'annonces\n", fontsize=50, color='#c0392b')
plt.title("Nombre d'annonces par Code Postal sur seloger.com\n", fontsize=60, color='#e74c3c')
plt.xticks(rotation= 45)
plt.tight_layout()

In [ ]:
prix = results[['price (€)','code_postal']].groupby('code_postal').mean().round().sort_values(by='price (€)', ascending=False)
prix.reset_index(0, inplace=True)
prix.head()


plt.figure(figsize=(50,25))
sns.barplot(x=prix['code_postal'], y=prix['price (€)'], palette="Blues_r")
plt.xlabel('\nCode Postal', fontsize=50, color='#2980b9')
plt.ylabel('Loyer moyen (en €)\n', fontsize=50, color='#2980b9')
plt.title("Loyer moyen par ville sur seloger.com\n", fontsize=60, color='#3742fa')
plt.xticks(rotation= 45)
plt.tight_layout()

In [ ]:
# Visualisation global de nos biens 
label = dfb.unique()
explode = (0.1, 0.1,0.1, 0.1, 0.5, 0.1)
popuratity = dfb.value_counts()
colors = ["#d45087", "#f95d6a","#ff7c43","#003f5c", "#a05195"]
plt.pie(popuratity, explode=explode, colors=colors,autopct='%1.1f%%')
plt.title("Nos différents bien en chiffre ",color="red")
plt.legend(loc='upper right', labels=label,frameon=True,fontsize=8)

In [ ]:
## Catégoriser nos valeurs en fonction d'ensemble
def discretize(price,limits):
    limits = sorted(limits)
    for s,e in zip(limits[:-1], limits[1:]):
        if s <= price < e:
            return f'{s}--{e}'.replace("inf", "\u221E")
    return "Unknown"

In [ ]:
# Définition des limites et application de la fonction
limits = [0,1000,2000,3000,4000,5000,8000,np.inf]
df_clean["price_categorie_m2"] = df_clean["price_m2"].apply(discretize,args=(limits,))

In [ ]:
df_clean["price_categorie_m2"].value_counts().sort_index().plot.pie(autopct='%1.1f%%', cmap="jet")
plt.title("Répartition des catégories de prix au m2",color="red")

In [ ]:
# Nombre de biens présent dans chaques catégorie
df_clean["price_categorie_m2"].value_counts()

In [ ]:
# Préparation des données pour la HeatMap
df_hm = df_clean.pivot_table(index="code_postal",columns="type",aggfunc="size",fill_value=0)

In [ ]:
# Résumé de nombre et du type de bien par ville
df_hm

In [ ]:
# Heatmap en fonction du code postal de chaque ville
sns.heatmap(df_hm)